In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from point2mn import point2mn
from point2mn import get_main_points
torch.manual_seed(1)

In [2]:
import numpy as np
import pandas as pd

In [3]:
embedding_index = {}

f = open('../../glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()

print('found word vecs: ',len(embedding_index))

found word vecs:  400000


In [184]:
main_points = pd.read_csv('../Meta Data/metadata_appended_main_points.csv') 
main_points.dropna(subset = ['For_Main_Points', 'against_Main_Points'], inplace = True)
main_points = main_points.reset_index(drop = True)
main_points.head()

,id,title,date,for,against,For_Main_Points,against_Main_Points
0,d20191112,Capitalism Is a Blessing,2019-11-12,"['John Mackey', 'Katherine Mangu-Ward']","['Bhaskar Sunkara', 'Richard D. Wolff']",['By promoting market competition and rewardin...,['Capitalism serves the interests of large cor...
1,d20191029,Parenting Is Overrated,2019-10-29,"['Robert Plomin', 'Nancy Segal']","['Paige Harden', 'Ann Pleshette Murphy']","[""We're in the midst of a DNA revolution: Whil...","['While DNA is important, factors like familia..."
2,d20191022,Europe Has Declared War on American Tech Compa...,2019-10-22,"['Roslyn Layton', 'Berin Szóka']","['Marietje Schaake', 'Ramesh Srinivasan']",['European regulators have declared war on Ame...,['Brussels isn’t waging war on Silicon Valley....
3,d20190917,Replace Private Insurance with Medicare for All,2019-09-17,"['Dr. Adam Gaffney', 'Joseph Sanberg']","['Nick Gillespie', 'Sally Pipes']",['The United States government should follow t...,['Individuals should have the freedom to choos...
4,d20190802,The Recent U.S. Policy Towards China Is Produc...,2019-08-02,"['Michael Pillsbury', 'Kori Schake']","['Graham Allison', 'Jake Sullivan']",['The prospect of China becoming an open and l...,['The United States and China are great compet...


In [185]:
results = pd.read_csv('../results_data/final_online.csv') 
results = results.loc[results['winner'].apply(lambda x: x != 'undecided')]
results['winner'] = results['winner'].apply(lambda x: 1 if x == 'for' else 0)
id2winner = results[['id', 'winner']].set_index('id').to_dict()['winner']

In [186]:
main_points['label'] = main_points.id.apply(lambda x: id2winner[x] if x in id2winner else np.nan)
main_points = main_points.dropna(subset = ['label']).reset_index(drop = True)
main_points.head()

,id,title,date,for,against,For_Main_Points,against_Main_Points,label
0,d20191112,Capitalism Is a Blessing,2019-11-12,"['John Mackey', 'Katherine Mangu-Ward']","['Bhaskar Sunkara', 'Richard D. Wolff']",['By promoting market competition and rewardin...,['Capitalism serves the interests of large cor...,0.0
1,d20191029,Parenting Is Overrated,2019-10-29,"['Robert Plomin', 'Nancy Segal']","['Paige Harden', 'Ann Pleshette Murphy']","[""We're in the midst of a DNA revolution: Whil...","['While DNA is important, factors like familia...",1.0
2,d20191022,Europe Has Declared War on American Tech Compa...,2019-10-22,"['Roslyn Layton', 'Berin Szóka']","['Marietje Schaake', 'Ramesh Srinivasan']",['European regulators have declared war on Ame...,['Brussels isn’t waging war on Silicon Valley....,0.0
3,d20190917,Replace Private Insurance with Medicare for All,2019-09-17,"['Dr. Adam Gaffney', 'Joseph Sanberg']","['Nick Gillespie', 'Sally Pipes']",['The United States government should follow t...,['Individuals should have the freedom to choos...,0.0
4,d20190802,The Recent U.S. Policy Towards China Is Produc...,2019-08-02,"['Michael Pillsbury', 'Kori Schake']","['Graham Allison', 'Jake Sullivan']",['The prospect of China becoming an open and l...,['The United States and China are great compet...,0.0


In [187]:
inputs = main_points.id.tolist()
labels =  main_points.label.tolist()
# labels = torch.tensor(labels)

In [161]:
from sklearn.model_selection import train_test_split

In [210]:
X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size = 0.2)

In [211]:
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)


In [212]:
len(X_train)

95

In [213]:
len(X_test)

24

In [168]:
EMBEDDING_DIM = 200
HIDDEN_DIM = 200
OUTPUT_SIZE = 1
N_LAYERS = 1
# all should use the same one lstm layer
lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM)


Hidden state initialize differently every time running LSTM??



In [190]:
def run_lstm_one_sentence(sentence, title, lstm, embedding_index):
    inputs = point2mn(sentence, title, embedding_index)
    tensor_input = [torch.tensor([x]) for x in inputs]
    tensor_input = torch.cat(tensor_input).view(1, len(tensor_input), -1)
#     print(tensor_input.shape)
    hidden = (torch.randn(1, 1, HIDDEN_DIM), torch.randn(1, 1, HIDDEN_DIM)) #???????
    tensor_output, hidden = lstm(tensor_input, hidden)
    # should return hidden state and cell state in 'hidden' instead of output
    return hidden

In [191]:

def run_lstm_on_fid(fid, embedding_index, lstm, combine_func = torch.mean):
    """
    Given fid, find main points for the debate and for each sentence, pass corresponding 
    matching vectors to lstm and get hidden state in the end. Gather the hidden states
    in a list and do combine_func.
    -combine_func: the funtion applied to combine lstm outputs from one side elementwisely
    """
    
    title = main_points[main_points['id'] == fid].title.iloc[0]
    for_main_points, against_main_points = get_main_points(fid, main_points)
    for_output_list = []
    against_output_list = []
    
    for sentence in for_main_points:
        hidden_state, cell_state = run_lstm_one_sentence(sentence, title, lstm, embedding_index)
        for_output_list.append(hidden_state)
        
    for sentence in against_main_points:
        hidden_state, cell_state = run_lstm_one_sentence(sentence, title, lstm, embedding_index)
        against_output_list.append(hidden_state)
        
    if combine_func == torch.mean:
        for_torch = combine_func(torch.stack(for_output_list), dim = 0, keepdim = True)#[0]
        against_torch = combine_func(torch.stack(against_output_list), dim = 0, keepdim = True)#[0]
    else:
        for_torch = combine_func(torch.stack(for_output_list), dim = 0, keepdim = True)[0]
        against_torch = combine_func(torch.stack(against_output_list), dim = 0, keepdim = True)[0]
    return torch.cat((for_torch, against_torch), dim = 2)

In [172]:
# fid = 'd20191112'
# title = main_points[main_points['id'] == fid].title[0]
# fid_torch = run_lstm_on_fid(fid, embedding_index, lstm,  combine_func = torch.mean)

In [35]:
lstm_out = [run_lstm_on_fid(fid, embedding_index, lstm, combine_func = torch.mean) for fid in main_points.id]

In [44]:
torch.stack(lstm_out).contiguous().view(-1, HIDDEN_DIM).shape

torch.Size([154, 400])

In [240]:
class LSTMNet(nn.Module):
    def __init__(self, output_size, n_layers, embedding_dim, hidden_dim, drop_prob=0.5):
        super(LSTMNet, self).__init__()
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim * 2, output_size)
#         self.softmax = nn.Softmax(dim = 1)
        self.sigmoid = nn.Sigmoid()
        
#     def forward(self, x, hidden):
#     def forward(self, fids, hidden):
    def forward(self, fids):
        batch_size = len(fids)
#         x = x.long()
        lstm_out = torch.stack([run_lstm_on_fid(fid, embedding_index, self.lstm, combine_func = torch.max) \
                                for fid in fids])
    
#         lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, 2 * self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
#         out = self.softmax(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out#, hidden
    
#     def init_hidden(self, batch_size):
#         weight = next(self.parameters()).data
#         hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
#                       weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
#         return hidden

In [235]:
model = LSTMNet(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM)

In [236]:
lr = 0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [237]:
epochs = 10
counter = 0
print_every = 1000
clip = 5
valid_loss_min = np.Inf
train_acc = []
test_acc = []
model.train()
for i in range(epochs):
#     h = model.init_hidden(batch_size)
    
#     for inputs, labels in train_loader:
#         counter += 1
#         h = tuple([e.data for e in h])
#         inputs, labels = inputs.to(device), labels.to(device)
    model.zero_grad()
    output = model(X_train)
    loss = criterion(output.squeeze(), y_train.float())
    print(loss)
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    outs = model(X_train)
    train_acc.append(torch.sum((outs > 0.5) == y_train).item() / len(y_train))
    outs = model(X_test)
    test_acc.append(torch.sum((outs > 0.5) == y_test).item() / len(y_test))
    
    
#         if counter%print_every == 0:
#             val_h = model.init_hidden(batch_size)
#             val_losses = []
#             model.eval()
#             for inp, lab in val_loader:
#                 val_h = tuple([each.data for each in val_h])
#                 inp, lab = inp.to(device), lab.to(device)
#                 out, val_h = model(inp, val_h)
#                 val_loss = criterion(out.squeeze(), lab.float())
#                 val_losses.append(val_loss.item())
                
#             model.train()
#             print("Epoch: {}/{}...".format(i+1, epochs),
#                   "Step: {}...".format(counter),
#                   "Loss: {:.6f}...".format(loss.item()),
#                   "Val Loss: {:.6f}".format(np.mean(val_losses)))
#             if np.mean(val_losses) <= valid_loss_min:
#                 torch.save(model.state_dict(), './state_dict.pt')
#                 print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
#                 valid_loss_min = np.mean(val_losses)

tensor(0.6975, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6934, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6923, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6907, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6822, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6742, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6755, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6493, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6919, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6338, grad_fn=<BinaryCrossEntropyBackward>)


In [241]:
# train_acc

In [242]:
# test_acc

In [217]:
# outs = model(X_train)
# torch.sum((outs > 0.5) == y_train).item() / len(y_train)

0.4842105263157895

0.5416666666666666

In [233]:
pd.Series(y_test).value_counts(normalize = True)

1.0    0.541667
0.0    0.458333
dtype: float64

In [200]:
outs

tensor([0.5020, 0.3077, 0.3503, 0.5805, 0.4688, 0.5605, 0.1735, 0.3698, 0.3980,
        0.5669, 0.5357, 0.4020, 0.3885, 0.4056, 0.3149, 0.6157, 0.5548, 0.5643,
        0.3511, 0.5886, 0.3619, 0.5594, 0.4987, 0.6190],
       grad_fn=<SelectBackward>)

In [19]:
# class FullyNet(nn.Module):
#     def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
#         super(FullyNet, self).__init__()
#         self.output_size = output_size
#         self.n_layers = n_layers
#         self.hidden_dim = hidden_dim
        
#         #self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
#         self.dropout = nn.Dropout(drop_prob)
#         self.fc = nn.Linear(hidden_dim, output_size)
#         self.sigmoid = nn.Sigmoid()
        
#     def forward(self, x, hidden):
#         batch_size = x.size(0)
#         x = x.long()
#         #embeds = self.embedding(x)
#         lstm_out, hidden = self.lstm(embeds, hidden)
#         lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
#         out = self.dropout(lstm_out)
#         out = self.fc(out)
#         out = self.sigmoid(out)
        
#         out = out.view(batch_size, -1)
#         out = out[:,-1]
#         return out, hidden